## Leetcode 207 Course Schedule
> There are a total of numCourses courses you have to take, labeled from $0$ to $numCourses-1$. Some courses may have prerequisites, for example to take course $0$ you have to first take course $1$, which is expressed as a pair: $[0,1]$.  
> Given the total number of courses and a list of prerequisite pairs, is it possible for you to finish all courses?  ($1 <= numCourses <= 10^5$)  
> **Example:**  
> Input: numCourses = 3, prerequisites = [[0,1],[0,2],[1,2]]  
> Output: True  
> Explaination: You can finish courses in the order of 2,1,0  
> Input: numCourses = 3, prerequisites = [[0,1],[1,0]]]  
> Output: False


## Solution
> Construct a directed graph as follows: each course is node, each prerequisite $[i,j]$ is directed edge from $i$ to $j$. The problem is equivalent to detect if there is directed cycle in the graph. 
> To detect cycles, we start from an arbitrary node and perform a Depth First Search. If there exists a back edge, then a cycle is detected. If not, we detected a connected component. Continue until all nodes are visited.

In [9]:
def canFinish(numCourses, prerequisites):
    def DFS(node,edges):
        s = [node]
        visited={node:True}
        while len(s)>0:
            n = s.pop()
            if n in edges:
                for m in edges[n]:
                    if m==node:
                        return False,{}
                    if m not in visited:
                        s.append(m)
                        visited[m] = True
            print(s)
        return True,visited
    
    
    visited = {}
    edge_dict = {}
    for x in prerequisites:
        if x[0] in edge_dict:
            edge_dict[x[0]].append(x[1])
        else:
            edge_dict[x[0]] = [x[1]]
                
    for start in range(numCourses):
        if start not in visited:
            res, extra = DFS(start, edge_dict)
            if not res:
                return res
            else:
                visited.update(extra)
        print(start,visited)
    return True

In [64]:
def canFinish(numCourses, prerequisites):
    def DFS(edges, visited, node, has_cycle):
        '''
            DFS in recursion, if has_cycle detected, jump out of the recursion immediately
        '''
        if has_cycle[0]:
            return
        else:
            if node in edges:
                for k in edges[node]:
                    if visited[k]==2:   # node k already in current path
                        has_cycle[0] = True
                        return
                    else:
                        visited[k] = 2
                        DFS(edges, visited, k, has_cycle)
                        visited[k] = 1
            
    visited = {i:0 for i in range(numCourses)} # 0: nodes have not visited, 1: nodes visited, 2: nodes in current path
    edge_dict = {}
    for x in prerequisites:
        if x[0] in edge_dict:
            edge_dict[x[0]].append(x[1])
        else:
            edge_dict[x[0]] = [x[1]]
                
    for start in range(numCourses):
        if visited[start]==0:
            has_cycle = [False]
            DFS(edge_dict, visited, start, has_cycle)
        if has_cycle[0]:
            return False
    
    return True
    
    

In [5]:
canFinish(12, [[0,1],[1,2],[0,2],[5,6],[6,8],[8,7],[8,3],[10,2],[2,4],[4,8]])

True

In [10]:
canFinish(12, [[0,1],[1,2],[0,2],[5,6],[6,8],[8,7],[8,5],[10,2],[2,4],[4,8]])

[1, 2]
[1, 4]
[1, 8]
[1, 7, 5]
[1, 7, 6]
[1, 7]
[1]
[]
0 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True}
1 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True}
2 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True}
[]
3 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True}
4 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True}
5 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True}
6 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True}
7 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True}
8 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True}
[]
9 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True, 9: True}
[2]
[4]
[8]
[7, 5]
[7, 6]
[7]
[]
10 {0: True, 1: True, 2: True, 4: True, 8: True, 7: True, 5: True, 6: True, 3: True,

True

## Leetcode 210: Course Schedule II
> There are a total of $n$ courses you have to take labelled from $0$ to $n - 1$. Some courses may have prerequisites, for example, if prerequisites[i] = [ai, bi], this means you must take the course bi before the course ai.  
> Given the total number of courses numCourses and a list of the prerequisite pairs, return the ordering of courses you should take to finish all courses. If there are many valid answers, return any of them. If it is impossible to finish all courses, return an empty array.  
> **Example:**  
> Input: numCourses = 4, prerequisites = [[1,0],[2,0],[3,1],[3,2]]  
> Output: [0,2,1,3]

 



## Solution: 
> * topological sort problem, see https://en.wikipedia.org/wiki/Topological_sorting#:~:text=In%20computer%20science%2C%20a%20topological,before%20v%20in%20the%20ordering.
> * Here we use Kahn's algorithm: Find a list of "start nodes" which have no indegree edges and insert them into final set; Then update the indegree of the remainning nodes and repeat the process. The topological sort can be found after all nodes in the final set. If at some point we can't find nodes to insert, the graph must be cyclic.

In [72]:
def findOrder(numCourses, prerequisites):
    edge_dict = {}
    indeg = [0 for _ in range(numCourses)]
    for x in prerequisites:
        if x[1] in edge_dict:
            edge_dict[x[1]].append(x[0])
        else:
            edge_dict[x[1]]=[x[0]]
        indeg[x[0]] += 1
    
    res = []
    cand = []
    for k in range(numCourses):
        if indeg[k]==0:
            cand.append(k)
            
    while len(res)<numCourses:
        if len(cand)>0:
            res.extend(cand)
            new_cand = []
            for k in cand:
                if k in edge_dict:
                    for m in edge_dict[k]:
                        indeg[m] -= 1
                        if indeg[m]==0:
                            new_cand.append(m)
            cand = new_cand.copy()
            
        else:
            return []
        
    return res

In [74]:
findOrder(12, [[0,1],[1,2],[0,2],[5,6],[6,8],[8,7],[8,3],[10,2],[2,4],[4,8]])

[3, 7, 9, 11, 8, 6, 4, 5, 2, 1, 10, 0]

In [76]:
findOrder(12, [[0,1],[1,2],[0,2],[5,6],[6,8],[8,7],[8,0],[10,2],[2,4],[4,8]])

[]

## Leetcode 630: Course Schedule III
> There are $n$ different online courses numbered from $1$ to $n$. Each course has some duration(course length) $t$ and closed on $d$-th day. A course should be taken continuously for $t$ days and must be finished before or on the $d$-th day. You will start at the 1st day.  
> Given $n$ online courses represented by pairs $(t,d)$, your task is to find the maximal number of courses that can be taken.  
> The integer $1 <= d, t, n <= 10,000$.  
> **Example:**  
> Input: [[100, 200], [200, 1300], [1000, 1250], [2000, 3200]]  
> Output: 3


## Solution1: recursion
> Sort according to end time of the courses. The max number of days we got initially is the max of the course end times.   
> Each time, we consider the last course. If it can be take (duration < current_maxday), we can choose either take it or not. If we take it, the max number of days to finish the previous courses is updated to current_maxday - duration of the last course. If we don't take it, the keep the same current_maxday and consider courses before it.   
> We can write a recursion to compute the max number of courses that can be taken. The time complexity in the worst case is O(2^n).

In [84]:
def scheduleCourse(courses):
    courses_sorted = sorted(courses, key=lambda x:x[1])
    
    def get_max_courses(courses, deadline):
        if len(courses)==0:
            return 0
        else:
            effective_ddl = min(deadline, courses[-1][1])
            if courses[-1][0]<=effective_ddl: # can take the last course
                return max(get_max_courses(courses[0:-1], effective_ddl), 1+get_max_courses(courses[0:-1], effective_ddl-courses[-1][0]))
            else:
                return get_max_courses(courses[0:-1], effective_ddl)
            
    return get_max_courses(courses_sorted, courses_sorted[-1][1])

In [85]:
test = [[100, 200], [200, 1300], [1000, 1250], [2000, 3200]] 

In [86]:
scheduleCourse(test)

3

In [88]:
scheduleCourse([[100, 200], [200, 1300], [1000, 1250], [2000, 3200], [300,1600]])

4

## Solution 2: Dynamic programming
> Similarly sort courses according to their ending time. 
> Use $dp[i][t]$ to denote the max number of courses one can take from the first i courses with max course ending time t. Let c be the vector of duration and deadline of course i, then the recursion is  
> $dp[i][t] = \max\{dp[i-1][t], 1 + dp[i-1][\min(t,c[1])-c[0]] \}$ if $\min(t, c[1])>=c[0]$ and $dp[i][t] = dp[i-1][t]$ otherwise. 
> Time complexity is $O(nd)$

In [115]:
def scheduleCourse(courses):
    courses_sorted = sorted(courses, key=lambda x:x[1])
    n = len(courses)
    maxt = courses_sorted[-1][1]
    
    dp = [[0 for _ in range(maxt+1)]]
    
    for i in range(n):
        c = courses_sorted[i]
        l = []
        for t in range(maxt+1):
            if min(t, c[1])<c[0]:
                l.append(dp[i][t])
            else:
                l.append(max(dp[i][t], 1+dp[i][min(t,c[1])-c[0]]))
        dp.append(l)
    return dp[n][maxt]

In [116]:
scheduleCourse([[1,2],[2,3]])

2

In [117]:
scheduleCourse([[100, 200], [200, 1300], [1000, 1250], [2000, 3200], [300,1600]])

4

## Solution 3: Greedy
> First, **the courses in the optimal solution must appear in ascending order with respect to their end time.** Otherwise, if $e_a>e_b$ but a is before b, then we can flip a and b (ending time of b will be earlier, ending time of a = current ending time of b < e_b < e_a so it is still valid).  
> Use the above statement, we sort all the courses by their ending time e. We maintain the current feasible set of courses and consider one more course at a time from left to right.  
> If at some point we cannot add the new course (only need to check if current end time + new course duration > new course end time), then one course must be deleted. We greedily delete the one with the longest duration. Reasons are:
> 1. The remaining course can be taken since i). orginal courses before that course has exactly the same end time; ii). original courses after that course has ealier end time than before; iii). since the max duration has been replaced with the new course duration, the overall finish time must be earlier than original ending time, which is earlier than the new course end time. 
> 2. By removing the longest duration course to eliminate conflicts, we ensure that we have saved most time for the remaining courses, that is, **the remaining courses can start at an earliest possible time.**

> To effectively compute the course with the longest duration in the list, we maintain a maxheap. Time complexity of the algorithm is $O(n\log n)$.

In [121]:
import heapq as hq
hq.heappushpop?

Signature: hq.heappushpop(heap, item, /)
Docstring:
Push item on the heap, then pop and return the smallest item from the heap.

The combined action runs more efficiently than heappush() followed by
a separate call to heappop().
Type:      builtin_function_or_method


In [122]:
import heapq as hq
def scheduleCourse(courses):
    courses_sorted = sorted(courses, key=lambda x:x[1])
    cand_duration = []
    maxday = 0
    n = len(courses)
    
    for c in courses_sorted:
        if c[0]+maxday > c[1]: # cannot schedule course c
            maxduration = -hq.heappushpop(cand_duration, -c[0])
            maxday = maxday - maxduration + c[0]
        else:
            maxday = maxday + c[0]
            hq.heappush(cand_duration, -c[0])
    return len(cand_duration)

In [123]:
scheduleCourse([[100, 200], [200, 1300], [1000, 1250], [2000, 3200], [300,1600]])

4

In [ ]:
Leetcode 300